# Installing the required libraries

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


c:\Users\User\OneDrive\Desktop\work\llm_hackathon\.venv\Scripts\python.exe: No module named pip


# Importing relevant libraries

In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from typing import Optional
from pydantic import BaseModel, Field
import getpass
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools.render import render_text_description # to describe tools as a string 
from langchain_core.output_parsers import JsonOutputParser # ensure JSON input for tools
from operator import itemgetter # to retrieve specific items in our chain.

# Setting up tools

In [26]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Draw, rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps

from pydantic import BaseModel, Field, field_validator
from enum import Enum
from typing import Annotated

# --- Define tools ---
class MultiplyInput(BaseModel):
    a: int = Field(..., description="first integer")
    b: int = Field(..., description="second integer")

@tool("multiply", args_schema=MultiplyInput)
def multiply(a: int, b: int) -> int:
    """Multiply a with b"""
    return a * b

class AddInput(BaseModel):
    x: int = Field(..., description="first integer")
    y: int = Field(..., description="second integer")

@tool("add", args_schema=AddInput)
def add(x: int, y: int) -> int:
    """Add x and y"""
    return x + y

@tool
def converse(input: str) -> str:
    "Provide a natural language response using the user input."
    return llm.invoke(input)

# Pydantic stuff i dont like this >.<
DESC_MAP = {name: fn for name, fn in Descriptors.descList}
DESC_NAMES = sorted(DESC_MAP.keys())

class DescriptorArgs(BaseModel):
    smiles: str = Field(..., description="SMILES string of the molecule.")

    # Pylance sees 'str', but the LLM will see an enum in the schema
    descriptor: Annotated[
        str,
        Field(
            description=f"RDKit descriptor name. One of: {', '.join(DESC_NAMES[:50])} … (total {len(DESC_NAMES)})",
            json_schema_extra={"enum": DESC_NAMES},
        ),
    ]

    # Runtime validation (and optional case-insensitive normalization)
    @field_validator("descriptor")
    @classmethod
    def validate_descriptor(cls, v: str) -> str:
        v2 = v.strip()
        if v2 in DESC_MAP:
            return v2
        # case-insensitive fallback
        matches = [n for n in DESC_MAP if n.lower() == v2.lower()]
        if matches:
            return matches[0]
        raise ValueError(f"Invalid descriptor '{v}'. Choose one of: {', '.join(DESC_NAMES[:20])} …")

# TOOLS 
@tool(args_schema=DescriptorArgs)
def descriptor_calculation(smiles: str, descriptor: str) -> str:
    """Calculate molecular descriptors from a SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string"
    
    desc = {desc_name: func(mol) for desc_name, func in Descriptors.descList}
    return desc[descriptor]
    
@tool#(args_schema=DescriptorArgs)
def MolToSmiles(molecule: str) -> str:
    """Validate and canonicalize a SMILES string."""
    candidate = molecule.strip()
    mol = Chem.MolFromSmiles(candidate)
    
    if mol is None:
        raise ValueError(f"Invalid SMILES: {candidate}")
    return Chem.MolToSmiles(mol, canonical=True)


@tool
def partial_charge_calculation(smiles: str) -> str:
    """Calculate Gasteiger partial charges from a SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string"

    AllChem.Mol.ComputeGasteigerCharges(mol)
    charges = [atom.GetProp('_GasteigerCharge') for atom in mol.GetAtoms()]
    return str(charges)


@tool
def generate_png_descriptors(smiles: str) -> str:
    """Generate a 2D depiction of the molecule with descriptors annotated."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string"
    
    AllChem.Mol.ComputeGasteigerCharges(mol)
    contribs = [mol.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge') for i in range(mol.GetNumAtoms())]
    d2d = Draw.MolDraw2DCairo(400, 400)
    sim = SimilarityMaps.GetSimilarityMapFromWeights(mol, contribs, d2d, colorMap='jet', contourLines=10)
    sim.FinishDrawing()
    sim.WriteDrawingText('molecule.png')

    return "molecule.png"

@tool
def display_image(file_path: str) -> str:
    """Display the image from the file path."""
    from PIL import Image
    img = Image.open(file_path)
    img.show()
    return "Image displayed"


@tool
def substructure_search(molecule_smiles: str, pattern: str, use_chirality: bool = False) -> str:
    """Search for substructures in a molecule using SMILES or SMARTS patterns.
    
    Args:
        molecule_smiles: SMILES string of the target molecule to search in
        pattern: SMILES or SMARTS pattern to search for (e.g., 'ccO', 'CO', 'c[NH1]')
        use_chirality: Whether to consider stereochemistry in matching (default: False)
    
    Returns:
        Information about substructure matches including whether pattern is found,
        atom indices of matches, and total number of matches found.
    """
    try:
        mol = Chem.MolFromSmiles(molecule_smiles)
        if mol is None:
            return f"Invalid molecule SMILES: {molecule_smiles}"
        
        # Try SMARTS first, then SMILES
        patt = Chem.MolFromSmarts(pattern) or Chem.MolFromSmiles(pattern)
        if patt is None:
            return f"Invalid pattern: {pattern}"
        
        # Check for matches
        has_match = mol.HasSubstructMatch(patt, useChirality=use_chirality)
        
        if not has_match:
            return f"Pattern '{pattern}' not found in {molecule_smiles}"
        
        # Get all matches
        matches = mol.GetSubstructMatches(patt, useChirality=use_chirality)
        
        result = [f"Pattern '{pattern}' found {len(matches)} time(s) in {molecule_smiles}"]
        result.append(f"Match indices: {matches}")
        
        return "\n".join(result)
        
    except Exception as e:
        return f"Error: {str(e)}"
    


@tool
def chemical_transformation_remove(smiles: str, smarts: str):
    """Remove SMARTS from SMILES
    
    Args:
        smiles: molecule string to remove molecule from.
        smarts: The molecule pattern to remove. 

    Returns:
        The modified molecules in SMILES format.
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return f"Invalid molecule SMILES: {smiles}"
        
        # Try SMARTS first, then SMILES
        patt = Chem.MolFromSmarts(smarts) or Chem.MolFromSmiles(smarts)
        if patt is None:
            return f"Invalid pattern: {smarts}"
        
        rm = Chem.DeleteSubstructs(mol,patt)
        return "Modified Molecule: "+ Chem.MolToSmiles(rm)
    except Exception as e:
        return f"Error: {e}"
    

@tool
def chemical_transformation_replace(smiles: str, smarts: str, replace: str):
    """Remove SMARTS from SMILES
    
    Args:
        smiles: molecule string to remove molecule from.
        smarts: The molecule pattern to remove.
        replace: The molecule pattern to replace with. 

    Returns:
        The modified molecules in SMILES format.
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return f"Invalid molecule SMILES: {smiles}"
        
        replace = Chem.MolFromSmiles(replace)
        if mol is None:
            return f"Invalid molecule SMILES: {replace}"
        
        # Try SMARTS first, then SMILES
        patt = Chem.MolFromSmarts(smarts) or Chem.MolFromSmiles(smarts)
        if patt is None:
            return f"Invalid pattern: {smarts}"
        
        mod_m = Chem.ReplaceSubstructs(mol,patt,replace)
        return "Modified Molecule: "+ Chem.MolToSmiles(mod_m)
    except Exception as e:
        return f"Error: {e}"


@tool
def molecule_side_chain_removal(smiles: str, smiles_sc: str):
    """
    Replaces sidechains with the provided side chain
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return f"Invalid molecule SMILES: {smiles}"
        
        sc = Chem.MolFromSmiles(smiles_sc)
        if sc is None:
            return f"Invalid molecule SMILES: {smiles_sc}"

        
        mod_m = Chem.ReplaceSidechains(smiles, smiles_sc)
        return "Modified Molecule: "+ Chem.MolToSmiles(mod_m)
    except Exception as e:
        return f"Error: {e}"

@tool
def molecule_core_removal(smiles: str, core: str):
    """
    Replaces core with the provided core
    """
    
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return f"Invalid molecule SMILES: {smiles}"
    
        core_ = Chem.MolFromSmiles(core)
        if core_ is None:
            return f"Invalid molecule SMILES: {core}"
        
        mod_m = ReplaceCore(mol, core_)
        return "Modified Molecule: " + Chem.MolToSmiles(mod_m)
    except Exception as e:
        return f"Error: {e}"
    


@tool
def molecular_structure_analysis(smiles: str) -> str:
    """Analyze molecular structure by looping over atoms and bonds from a SMILES string.
    
    Args:
        smiles: A SMILES string representing a molecule (e.g., 'C1OC1', 'c1ccccc1', 'CCO')
    
    Returns:
        Detailed information about atoms, bonds, and their relationships including
        atomic numbers, symbols, bond types, connectivity, neighbor relationships,
        atom indices and valences.
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return "Invalid SMILES string. Please provide a valid SMILES representation."
        
        result = []
        result.append(f"Molecular formula: {rdMolDescriptors.CalcMolFormula(mol)}")
        result.append(f"Number of atoms: {mol.GetNumAtoms()}")
        result.append(f"Number of bonds: {mol.GetNumBonds()}")
        result.append("")
        
        # Loop over atoms
        result.append("ATOMS:")
        for i, atom in enumerate(mol.GetAtoms()):
            symbol = atom.GetSymbol()
            atomic_num = atom.GetAtomicNum()
            valence = atom.GetExplicitValence()
            neighbors = [n.GetSymbol() for n in atom.GetNeighbors()]
            neighbor_indices = [n.GetIdx() for n in atom.GetNeighbors()]
            
            result.append(f"Atom {i}: {symbol} (atomic_num={atomic_num}, valence={valence})")
            result.append(f"  Neighbors: {neighbors} at indices {neighbor_indices}")
        
        result.append("")
        
        # Loop over bonds
        result.append("BONDS:")
        for i, bond in enumerate(mol.GetBonds()):
            begin_idx = bond.GetBeginAtomIdx()
            end_idx = bond.GetEndAtomIdx()
            bond_type = bond.GetBondType()
            begin_symbol = mol.GetAtomWithIdx(begin_idx).GetSymbol()
            end_symbol = mol.GetAtomWithIdx(end_idx).GetSymbol()
            
            result.append(f"Bond {i}: {begin_symbol}({begin_idx})-{end_symbol}({end_idx}) [{bond_type}]")
        
        result.append("")
        
        # Additional connectivity information
        result.append("CONNECTIVITY MATRIX:")
        for i in range(mol.GetNumAtoms()):
            connections = []
            for j in range(mol.GetNumAtoms()):
                bond = mol.GetBondBetweenAtoms(i, j)
                if bond:
                    connections.append(f"{j}({bond.GetBondType()})")
            if connections:
                symbol = mol.GetAtomWithIdx(i).GetSymbol()
                result.append(f"Atom {i}({symbol}) connected to: {', '.join(connections)}")
        
        return "\n".join(result)
        
    except Exception as e:
        return f"Error analyzing molecule: {str(e)}"



tools = [multiply, add, partial_charge_calculation, generate_png_descriptors, descriptor_calculation, molecular_structure_analysis, molecule_core_removal, molecule_side_chain_removal, chemical_transformation_replace, chemical_transformation_remove, substructure_search]

tools_by_name: dict[str, any] = {t.name: t for t in tools}


# setting up the LLM

In [ ]:
#config
#if not os.environ.get("GOOGLE_API_KEY"):
#  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#google_api_key = os.environ.get("GOOGLE_API_KEY")
#if not google_api_key:
#    raise ValueError("GOOGLE_API_KEY environment variable not set. Please set it in the .env file or your environment.")

def tool_chain(model_output):
    print([x for x in model_output])
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

llm = OllamaLLM(model="llama3.1:8b")
query = input("Input: ")

rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}
Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
The value associated with the 'arguments' key should be a dictionary of parameters."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | llm | JsonOutputParser() | tool_chain # pipeline the prompt into llm's template then pass the output to json parser then pass it to tool_chain

multi_system_prompt = f"""You have these tools:

{rendered_tools}

You will solve the user's task by choosing tools only as ABSOLUTELY neccessary and arguments. If you have enough information to answer the user's question after a tool call, do not call any more tools.
Return ONLY JSON with keys:
- "name": tool name to call next, or "final" to finish
- "arguments": dict of arguments for that tool (omit if name == "final")
- "final_answer": present ONLY if name == "final"

Use prior tool results (provided in the scratchpad) to decide the next step.
"""

multi_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", multi_system_prompt),
        ("user", "{input}"),
        ("assistant", "{scratchpad}"), 
    ]
)

multi_chain = multi_prompt | llm | JsonOutputParser()

def run_multi_step(user_query: str, max_steps: int = 128):
    scratchpad = ""
    for _ in range(max_steps):
        msg = multi_chain.invoke({"input": user_query, "scratchpad": scratchpad})
        step = msg if msg.isinstance(msg, dict) else {}
        name = step["name"]
        print(step)
        if name == "final":
            return step.get("final_answer", "(no final answer)")
        if name not in tools_by_name:
            raise ValueError(f"Unknown tool: {name}")
        args = step.get("arguments", {}) or {}
        tool_result = tools_by_name[name].invoke(args)
        scratchpad += f"\nTOOL={name} ARGS={args} RESULT={tool_result}"
    raise RuntimeError("Max steps reached without final answer")

final_answer = run_multi_step(query)
print(query)
print(final_answer)

#chain = llm
#chain.invoke(query)

{'name': 'descriptor_calculation', 'arguments': {'smiles': 'C1=CC=C(C(=C1)C(=O)O)C(=O)O', 'descriptor': 'TPSA'}}
{'name': 'final', 'arguments': {}, 'final_answer': 74.6}
what is the TPSA value for benzoic acid
74.6
